# Extended Kalman Filter
* Recursive estimator(predictor-corrector) that solves the *linear quadratic estimation problem* using Minimum Mean Squared Error (MSSE) method

\begin{equation*}
\hat{x}_{i|j} = arg min_{\hat{x}_{i|j}\epsilon R^n}E[(x_i - \hat{x}_{i|j})(x_i - \hat{x}_{i|j})^T | z_1, ..., z_j]
\end{equation*}

$\hat{x}_{i|j}$: state estimate for time i given all the measurements up to time j

# Discrete-Time Non-Linear System Models


## Process model

\begin{equation*}
x_k = f(x_{k-1}, u_k, w_k)
\end{equation*}

\begin{equation*}
z_k = h(x_k, v_k)
\end{equation*}

$x_k$: State Vector

$u_k$: Control Input Vector

$w_k$: Process Model Noise Vector

$v_k$: Measurement Noise Vector

Assumptions:

- Gaussian distribution with zero mean and given covariance matrix
  
$w_k \sim N(0, Q_k)$
$v_k \sim N(0, R_k)$


## Prediction step

\begin{equation*}
\hat{x}^-_k = f(\hat{x}^+_{k-1}, u_k, 0)
\end{equation*}

\begin{equation*}
{ \hat{x}_0 = \begin{bmatrix} p_{x_{gps}} \\ p_{y_{gps}} \\ 0 \\ 0 \end{bmatrix}}
\end{equation*}

\begin{equation*}
{ P_0 = \begin{bmatrix} \sigma^2_{gps} & 0 & 0 & 0 \\ 0 & \sigma^2_{gps} & 0 & 0 \\ 0 & 0 & \sigma^2_{\psi_0} & 0 \\ 0 & 0 & 0 & \sigma^2_{V_0} \end{bmatrix}}
\end{equation*}

- Assume

\begin{equation*}
{ \hat{x} = \begin{bmatrix} p_x \\ p_y \\ \psi \\ V \end{bmatrix}}
\end{equation*}

\begin{equation*}
{ \begin{bmatrix} p_x \\ p_y \\ \psi \\ V \end{bmatrix}_k = \begin{bmatrix} p_x \\ p_y \\ \psi \\ V \end{bmatrix}_{k-1} + \Delta t \begin{bmatrix} V_{k-1} cos(\psi_{k-1}) \\ V_{k-1} sin(\psi_{k-1}) \\ \dot{\psi_k} \\ a_k \end{bmatrix}}
\end{equation*}

- For Covariance,

\begin{equation*}
P^-_k = \nabla f_x p^+_{k-1} \nabla f^T_x + \nabla f_w Q_k \nabla f^T_w
\end{equation*}


\begin{equation*}
F_k = \nabla f_x = \begin{bmatrix} 1 & 0 & - \Delta t V_{k-1} sin(\psi_{k-1}) & \Delta t cos(/psi_{k-1}) \\ 0 & 1 & \Delta t V_{k-1} cos(\psi_{k-1}) & \Delta t sin(\psi_{k-1}) \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}
\end{equation*}

- For Gyro sensor, 
  
\begin{equation*}
Q = \begin{bmatrix} 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & \Delta t^2 \sigma^2_{gyro} & 0 \\ 0 & 0 & 0 & \Delta t^2 \sigma^2_{accel} \end{bmatrix}
\end{equation*}

\begin{equation*}
P^-_k = F_k P^+_{k-1} F^T_k + Q
\end{equation*}

## Update step

Innovation vector
\begin{equation*}
\nu_k = z_k - h(\hat {x}^-_k, 0) 
\end{equation*}

* Assume
\begin{equation*}
{ \hat{x} = \begin{bmatrix} p_x \\ p_y \\ \psi \\ V \end{bmatrix}}
\end{equation*}

* For lidar, 

\begin{equation*}
\begin{bmatrix} \hat{r} \\ \hat{\theta} \end{bmatrix} = \begin{bmatrix} \sqrt{ (L_x - \hat{p_x})^2 + (L_y - \hat{p_y})^2} \\ \arctan(\frac{L_y - \hat{p_y}}{L_x - \hat{p_x}}) - \hat{\psi} \end{bmatrix}
\end{equation*}

Innovation Covariance
\begin{equation*}
S_k = \nabla h_x P^-_k \nabla h^T_x + \nabla h_{\nu} R_k \nabla h^T_{\nu}
\end{equation*}

where the Jacobians are
\begin{equation*}
\nabla h_x = \frac {\delta h}{\delta x} \vert_{x = \hat{x}^-_k}
\end{equation*}

\begin{equation*}
\nabla h_{\nu} = \frac {\delta h}{\delta {\nu}} \vert_{x = \hat{x}^-_k}
\end{equation*}

Jacobian Matrix 
\begin{equation*}
H = \nabla h_x = \begin{bmatrix} \frac{1}{d} (\hat{p_x} - L_x) & \frac{1}{d} (\hat{p_y} - L_y) & 0 & 0 \\ \frac{-1}{d^2} (\hat{p_y} - L_y) & \frac{1}{d^2} (\hat{p_x} - L_x) & -1 & 0 \end{bmatrix}
\end{equation*}
\begin{equation*}
d = \sqrt{(L_x - \hat{p_x})^2 + (L_y - \hat{p_y})^2}
\end{equation*}

* For lidar, 
\begin{equation*}
\nu = \begin{bmatrix} r \\ \theta \end{bmatrix}_{meas} - \begin{bmatrix} \hat{r} \\ \hat{\theta} \end{bmatrix}
\end{equation*}

\begin{equation*}
R = \begin{bmatrix} \sigma^2_r & 0 \\ 0 & \sigma^2_{\theta} \end{bmatrix}
\end{equation*}


\begin{equation*}
S_k = \nabla h_x P^-_k \nabla h^T_x + R
\end{equation*}

Kalman Gain
\begin{equation*}
K_k = P^-_k \nabla h^T_x S^{-1}_k
\end{equation*}

State Estimate
\begin{equation*}
\hat{x}^+_k = \hat{x}^-_k + K_k \nu_k
\end{equation*}

Covariance Estimate
\begin{equation*}
P^+_k = (I - K_k \nabla h_x)P^-_k
\end{equation*}
